In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import mysql.connector
import networkx as nx
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon,MultiPolygon,LineString,MultiLineString,LinearRing,Point
from shapely import symmetric_difference
from shapely.ops import nearest_points
from shapely.wkt import loads
import scipy
import time
import numpy as np
import random
from networkx.drawing.nx_pydot import graphviz_layout
import matplotlib.pyplot as plt
import osmnx as ox
import glob
from itertools import combinations,product
from copy import deepcopy
import os
from lib_contour_lines import *
import mysql.connector


In [2]:
#set global net_buffer_length=1000000; 
#set global max_allowed_packet=1000000000;

In [5]:

cnx = mysql.connector.connect(user='spirz', 
                              password='this_is_my_PASSWORD_m8',
                              host='localhost',
                              database='dem_from_contours_lines',
                              autocommit=True)
cursor = cnx.cursor(buffered=True,dictionary=True)
contours_lines_table_name='contours_lines'
tree_edges_table_name="tree_edges"
2

2

In [4]:
DELETE_TABLES=False
if DELETE_TABLES:
    try:
        cmd="""DROP TABLE `%s`"""%tree_edges_table_name
        cursor.execute(cmd)
    except:
        pass
    cmd="""DROP TABLE `%s`"""%contours_lines_table_name
    cursor.execute(cmd)
2

2

In [5]:
CREATE_TABLE=False
if CREATE_TABLE:
    cmd="""CREATE TABLE `%s`(
          `id` INT NOT NULL AUTO_INCREMENT,
          `source` INT,
          `is_closed` BOOLEAN NOT NULL,
          `elevation` FLOAT NOT NULL,
          `length` FLOAT,
          `number_points` FLOAT,
          `geometry` LINESTRING NOT NULL,
          PRIMARY KEY (id)
          )
          ENGINE=InnoDB;
        """%contours_lines_table_name
    cursor.execute(cmd)
2

2

In [6]:
#ASSUME GEODATAFRAMES ARE IN UTM COORDINATES
GENERATE=False
if GENERATE:
    t1=time.time()
    file_paths=glob.glob('COURBE_1-0__SHP_LAMB93_D02A_2021-01-01/COURBE/1_DONNEES_LIVRAISON_2021-01-01/COURBE_1-0_SHP_LAMB93_D02A_2021/*.shp')
    file_paths+=glob.glob('COURBE_1-0__SHP_LAMB93_D02B_2021-01-01/COURBE/1_DONNEES_LIVRAISON_2021-01-01/COURBE_1-0_SHP_LAMB93_D02B_2021/*.shp')

    for k,file_path in enumerate(file_paths[:]):
        print(file_path)
        df=gpd.read_file(file_path) 
        df['elevation']=df['ALTITUDE']
        add_contour_lines_to_database(df,cursor,source=k)
        t3=time.time()
        cmd="""SELECT c1.id AS id_1,ST_asText(c1.geometry) AS border_1,c2.id AS id_2,ST_asText(c2.geometry) AS border_2 FROM
        (SELECT * FROM %s WHERE source!=%i) AS c1
        JOIN (SELECT * FROM %s WHERE source=%i) AS c2
        ON c1.elevation=c2.elevation AND c1.length=c2.length
        
        ;
        
        """%(contours_lines_table_name,k,contours_lines_table_name,k)
        cursor.execute(cmd)
        L=cursor.fetchall()
        t4=time.time()
        print('getting pairs took %f'%(t4-t3))
        bad_ids=[elem['id_2'] for elem in L if loads(elem['border_1'])==loads(elem['border_2'])]

        t5=time.time()
        print('comparing pairs took %f'%(t5-t4))
        if len(bad_ids)>0:
            bad_ids_string='(%s)'%', '.join([str(bad_id) for bad_id in bad_ids])
            cmd="DELETE FROM %s WHERE id IN %s"%(contours_lines_table_name,bad_ids_string)
            cursor.execute(cmd)
            t6=time.time()
            print('deleting took %f'%(t6-t5))


        


    t2=time.time()
    print(t2-t1)
2

2

In [7]:
t1=time.time()
cmd="SELECT DISTINCT elevation FROM %s ORDER BY elevation"%contours_lines_table_name
cmd=cursor.execute(cmd)
elevations=np.array([elem['elevation'] for elem in cursor.fetchall()])
t2=time.time()
print(t2-t1)

0.10340261459350586


In [8]:
MERGE=False
if MERGE:
    t1=time.time()
    for level in elevations:
        print(level)
        level_open_contours_df=get_level_contours_df(cursor,level,contours_lines_table_name,is_closed=False)
        if level_open_contours_df is not None:
            crs=level_open_contours_df.estimate_utm_crs()
            level_open_contours_df=level_open_contours_df.to_crs(crs)
            merged_contours_lines=get_contour_lines_from_elevation_df(level_open_contours_df)
            if len(merged_contours_lines)>0:
                merged_contours_df=gpd.GeoDataFrame([{'geometry':ls} for ls in merged_contours_lines],geometry='geometry',crs=crs)
                merged_contours_df['elevation']=level
                add_contour_lines_to_database(merged_contours_df,cursor,is_closed=True)

                cursor.execute(cmd)
    t2=time.time()
    print(t2-t1)
2

2

In [9]:
DELETE_EDGE_TABLE=False
if DELETE_EDGE_TABLE:
    cmd="""DROP TABLE `%s`"""%tree_edges_table_name
    cursor.execute(cmd)
2

2

In [10]:
COMPUTE_TREE=False
tree_edges_table_name="tree_edges"

if COMPUTE_TREE:
    t1=time.time()
    edges=[]
    for k,level in enumerate(elevations):
        print(level)
        nearby_levels=[level]
        if k!=0:
            nearby_levels.append(elevations[k-1])
        if k!=len(elevations)-1:
            nearby_levels.append(elevations[k+1])
            
        polys_df=get_level_contours_df(cursor,level,contours_lines_table_name,is_closed=True)
        if polys_df is not None:
            polys_df['geometry']=polys_df['geometry'].apply(lambda ls:Polygon(ls))
            crs=polys_df.estimate_utm_crs()
            polys_df=polys_df.to_crs(crs)
            polys_df['area']=polys_df['geometry'].apply(lambda poly:poly.area)

            points_dfs=[get_level_contours_df(cursor,nearby_level,contours_lines_table_name,is_closed=True) for nearby_level in nearby_levels]
            points_dfs=[df for df in points_dfs if df is not None]
            if len(points_dfs)>0:
                points_df=pd.concat(points_dfs)
                points_df=points_df.to_crs(crs)
                points_df['area']=points_df['geometry'].apply(lambda ls:Polygon(ls).area)
                points_df['geometry']=points_df['geometry'].apply(lambda ls:ls.representative_point())

                inter=gpd.overlay(polys_df,points_df,keep_geom_type=False)
                inter=inter[inter.id_1!=inter.id_2]
                
                edges+=list(zip(inter['id_1'],inter['id_2'],inter['area_1']-inter['area_2']))

    G=nx.DiGraph()
    G.add_edges_from([(u,v,{'weight':area_diff}) for u,v,area_diff in edges])
    for node in G.nodes():
        in_edges=[(u,v,d) for u,v,d in G.in_edges(node,data=True) if u!=v]
        if len(in_edges)>1:
            in_edges=sorted(in_edges,key=lambda edge:edge[2]['weight'])
            for u,v,_ in in_edges[1:]:
                G.remove_edge(u,v)
    tree_edges=G.edges()

    cmd="""CREATE TABLE `%s`(
    `begin` INT NOT NULL,
    `end` INT NOT NULL,
    FOREIGN KEY (begin) REFERENCES %s(id),
    FOREIGN KEY (end) REFERENCES %s(id) ,
    PRIMARY KEY (begin,end)
    );
        """%(tree_edges_table_name,contours_lines_table_name,contours_lines_table_name)
    
    cursor.execute(cmd)
    
    string_list=',\n'.join(['(%i,%i)'%(u,v) for (u,v) in tree_edges])
    cmd="""INSERT INTO %s (`begin`,`end`) VALUES %s;"""%(tree_edges_table_name,string_list)
    cursor.execute(cmd)
    
    t2=time.time()
    print(t2-t1)

In [11]:
t1=time.time()
G=build_graph(cursor)
t2=time.time()
print(t2-t1)

0.578125


In [12]:
t1=time.time()
place_name='Corse, France'
G_osm=ox.graph_from_place(place_name,network_type='drive')
G_osm=to_multi_graph(G_osm)
osm_crs=ox.graph_to_gdfs(G_osm,edges=False).estimate_utm_crs()
G_osm=ox.project_graph(G_osm,to_crs=osm_crs)
add_missing_geometries(G_osm)
t2=time.time()
print(t2-t1)

60.293384313583374


In [29]:
t1=time.time()
osm_edges_df=ox.graph_to_gdfs(G_osm,nodes=False,edges=True).explode()
osm_edges_df['edge']=osm_edges_df.index
osm_edges_df=osm_edges_df.loc[:,['edge','geometry']]
osm_edges_df['number_of_points']=osm_edges_df['geometry'].apply(lambda ls:len(ls.coords))
t2=time.time()
print(t2-t1)
osm_edges_df

/tmp/ipykernel_12266/1975615014.py:2: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  osm_edges_df=ox.graph_to_gdfs(G_osm,nodes=False,edges=True).explode()


0.867875337600708


edge  \
u           v           key                                       
14468319    15905397    0   0        (14468319, 15905397, 0, 0)   
            1434053206  0   0      (14468319, 1434053206, 0, 0)   
            1434053161  0   0      (14468319, 1434053161, 0, 0)   
15905397    1976904946  0   0      (15905397, 1976904946, 0, 0)   
            1976906939  0   0      (15905397, 1976906939, 0, 0)   
...                                                         ...   
9744995304  9744995311  0   0    (9744995304, 9744995311, 0, 0)   
            9744995310  0   0    (9744995304, 9744995310, 0, 0)   
9964974606  9964974609  0   0    (9964974606, 9964974609, 0, 0)   
9964974610  9964974611  0   0    (9964974610, 9964974611, 0, 0)   
11329589123 11329589133 0   0  (11329589123, 11329589133, 0, 0)   

                                                                        geometry  \
u           v           key                                                        
14468319    15905397    0   0  LINESTRING (525532.710 4599879.701, 525543.715...   
            1434053206  0   0  LINESTRING (525532.710 4599879.701, 525519.651...   
            1434053161  0   0  LINESTRING (525532.710 4599879.701, 525532.752...   
15905397    1976904946  0   0  LINESTRING (526252.098 4600660.017, 526286.419...   
            1976906939  0   0  LINESTRING (526252.098 4600660.017, 526253.327...   
...                                                                          ...   
9744995304  9744995311  0   0  LINESTRING (496541.756 4720170.475, 496521.473...   
            9744995310  0   0  LINESTRING (496541.756 4720170.475, 496558.928...   
9964974606  9964974609  0   0  LINESTRING (494174.708 4719764.250, 494163.668...   
9964974610  9964974611  0   0  LINESTRING (494100.379 4719512.527, 494038.489...   
11329589123 11329589133 0   0  LINESTRING (493598.201 4718833.478, 493587.298...   

                               number_of_points  
u           v           key                      
14468319    15905397    0   0                82  
            1434053206  0   0                 4  
            1434053161  0   0                20  
15905397    1976904946  0   0                12  
            1976906939  0   0                 7  
...                                         ...  
9744995304  9744995311  0   0                 2  
            9744995310  0   0                 7  
9964974606  9964974609  0   0                 4  
9964974610  9964974611  0   0                 2  
11329589123 11329589133 0   0                11  

[20792 rows x 3 columns]

In [30]:
osm_edges_df=osm_edges_df.sample(5000)
osm_edges_df

,,,,edge,geometry,number_of_points
u,v,key,,,,
2791064075,2791064077,0,0,"(2791064075, 2791064077, 0, 0)","LINESTRING (542201.554 4691778.082, 542191.110...",3
6269758052,6269758056,0,0,"(6269758052, 6269758056, 0, 0)","LINESTRING (534339.466 4687735.304, 534333.808...",7
5397422862,5397422863,0,0,"(5397422862, 5397422863, 0, 0)","LINESTRING (492753.864 4619609.217, 492716.082...",2
10600240021,10600240022,0,0,"(10600240021, 10600240022, 0, 0)","LINESTRING (482960.304 4711508.648, 482951.795...",3
1041170062,1041170161,0,0,"(1041170062, 1041170161, 0, 0)","LINESTRING (529625.903 4698026.798, 529631.442...",91
...,...,...,...,...,...,...
394580584,3486835877,0,0,"(394580584, 3486835877, 0, 0)","LINESTRING (478006.491 4644059.699, 478011.288...",31
335634038,2232151985,0,0,"(335634038, 2232151985, 0, 0)","LINESTRING (481488.608 4711106.868, 481476.266...",8
394858938,8292503730,0,0,"(394858938, 8292503730, 0, 0)","LINESTRING (486996.689 4648433.535, 487034.205...",8


In [42]:
t1=time.time()
intersection_ls=get_recursive_intersections(G,cursor,osm_edges_df,osm_crs)
t2=time.time()
print(t2-t1)

root 1.0
113762 2.0
14978 2
34299 2
36137 2
39181 2
39512 2
41133 8.0
43114 2
43185 2
44648 12.0
60834 2
113766 3.0
28511 3
34654 3
35905 6.0
35041 6.0
37320 3
37513 12.0
40072 3
47954 3
48435 3
113771 4.0
23281 4
28955 4
29319 16.0
24430 16.0
52456 4
66114 4
68056 4
113776 5.0
40570 5
41115 5
53135 11.0
59247 5
65063 5
113783 6.0
57519 6.0
67323 6
113789 7.0
17188 7
20582 18.0
26286 7.0
33851 7
35049 7.0
45793 16.0
46417 16.0
61884 7
61974 7
113792 8.0
10710 8.0
40760 8
41194 13.0
43623 13.0
45382 13
41914 63.0
44244 8
45039 8
45927 8.0
113797 9.0
21128 14.0
34942 11.0
45786 9
47033 11.0
113803 10.0
25772 10
43880 10.0
64460 10
113806 11.0
29368 11.0
34400 14.0
37902 11.0
39330 13.0
40362 11.0
44311 31.0
61462 11
66633 11
113811 12.0
18171 14.0
37957 17.0
38993 12.0
113817 13.0
13007 19.0
45127 35.0
54894 13.0
61153 13
113824 14.0
15531 14
22172 15.0
23704 15.0
39367 14
41533 14.0
42272 14
42758 21.0
45074 21.0
44277 14
46504 20.0
66630 17.0
59860 17.0
113830 15.0
42608 15
45145 28.0


In [46]:
intersection_ls

,elevation,depth,edge,geometry
0,0.0,1,"(2791064075, 2791064077, 0, 0)",POINT (542201.554 4691778.082)
1,0.0,1,"(2791064075, 2791064077, 0, 0)",POINT (542193.332 4691878.669)
2,0.0,1,"(6269758052, 6269758056, 0, 0)",POINT (534339.466 4687735.304)
3,0.0,1,"(6269758052, 6269758056, 0, 0)",POINT (534291.545 4687656.743)
4,0.0,1,"(5397422862, 5397422863, 0, 0)",POINT (492753.864 4619609.217)
...,...,...,...,...
363769,110.0,23,"(310478838, 5158333731, 0, 0)",POINT (522014.362 4608777.725)
363770,115.0,24,"(310478838, 5158333731, 0, 0)",POINT (521998.484 4608875.945)
363771,115.0,24,"(310478838, 5158333731, 0, 0)",POINT (522014.362 4608777.725)
363772,120.0,25,"(310478838, 5158333731, 0, 0)",POINT (522005.562 4608837.316)


In [47]:
intersection_ls['depth'].unique()

array([  1,   2,   3,   9,   4,   5,   6,   8,   7,  10,  15,  16,  12,
        13,  14,  11,  17,  34,  30,  31,  22,  23,  26,  21,  25,  19,
        32,  27,  29,  18,  24,  33,  20,  28,  35,  46,  43,  50,  49,
        41,  47,  44,  48,  36,  42,  38,  51,  37,  45,  40,  39,  52,
        54,  53,  77, 130,  64,  71,  59,  94,  92,  74, 101,  90,  81,
        63,  61, 119, 139,  96, 138,  72, 123, 104, 143,  88, 131, 148,
       144,  87,  85,  91,  93, 105,  75,  76, 111,  97, 132,  99, 120,
        66,  86, 134,  70, 135,  68,  69, 137, 122, 100,  58, 115,  82,
        98, 117, 147, 112, 114,  95, 145,  65, 129, 150, 142, 108,  56,
       151, 110,  57, 141,  89, 125, 102, 103, 140, 106, 121, 133, 107,
       152, 109, 149, 127, 154, 153,  79,  80, 146,  83, 128, 124,  67,
        84, 113,  78, 126,  55, 118, 156,  62, 155,  60, 116, 136,  73,
       157, 160, 158, 159, 161, 162, 163, 164, 165, 166, 167, 168, 169,
       170, 171, 172, 173, 174, 175, 176, 177, 195, 178, 179, 18

In [ ]:
254.8178539276123,13092